In [ ]:
%pip install mediapipe
%pip install opencv-python
%pip install numpy
%pip install keras
%pip install tensorflow
%pip install scikit-learn

In [2]:
import cv2 as cv
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
CATEGORIES = ["hello", "yes", "no", "iloveyou", "callme", "goodjob", "highfive", "dislike", "peace"]
NUM_CATS = len(CATEGORIES)

In [4]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()

In [5]:
def get_landmark_array(landmarks):
  arr = []
  for l in landmarks:
    arr.append([l.x, l.y, l.z])
  return np.array(arr)

In [6]:
def rotated(img, angle):
    h, w = img.shape[:2]
    rotation_matrix = cv.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    return cv.warpAffine(img, rotation_matrix, (w, h))

In [8]:
def load_train_data(data_dir):

    landmarks = []
    labels = []
    i = 0
    count = 0
    for label in CATEGORIES:
        print(label, end=" ")
        pictures = os.listdir(os.path.join(data_dir, label))
        for pic in pictures:
            img = cv.imread(os.path.join(data_dir, label, pic))
            


            imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            imgflipped = cv.flip(imgRGB, 1)
            imgRotated1 = rotated(imgRGB, 10)
            imgRotated2 = rotated(imgRGB, -10)
            imgZoom2p = cv.resize(imgRGB, None, fx=1.1, fy=1.1, interpolation=cv.INTER_LINEAR)
            imgZoomxp = cv.resize(imgRGB, None, fx=1.1, fy=1, interpolation=cv.INTER_LINEAR)
            imgZoomyp = cv.resize(imgRGB, None, fx=1, fy=1.1, interpolation=cv.INTER_LINEAR)
            imgZoomxn = cv.resize(imgRGB, None, fx=0.9, fy=1, interpolation=cv.INTER_LINEAR)
            imgZoomyn = cv.resize(imgRGB, None, fx=1, fy=0.9, interpolation=cv.INTER_LINEAR)
            imgZoom2n = cv.resize(imgRGB, None, fx=0.9, fy=0.9, interpolation=cv.INTER_LINEAR)
            imgZoom2xp = cv.resize(imgRGB, None, fx=0.8, fy=0.8, interpolation=cv.INTER_LINEAR)
            imgZoom2xn = cv.resize(imgRGB, None, fx=1.2, fy=1.2, interpolation=cv.INTER_LINEAR)


            AugmentedImages = [imgRGB, imgflipped, imgRotated1, imgRotated2, imgZoom2p, imgZoomxp, imgZoomyp, imgZoomxn, imgZoomyn, imgZoom2n, imgZoom2xp, imgZoom2xn]            

            for image in AugmentedImages:
                result = hands.process(image)
                if result.multi_hand_landmarks:
                    count += 1
                    landmarks.append(get_landmark_array(result.multi_hand_landmarks[0].landmark))
                    labels.append(i)
        i += 1

    print(f"\nTotal of {count} images generated")
    return (landmarks, labels)

In [9]:
def make_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input((21, 3, 1)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))

    # Output Layer
    model.add(tf.keras.layers.Dense(NUM_CATS, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [10]:
train_path = r"data"

In [ ]:
images, labels = load_train_data(train_path)

In [12]:
labels = tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=0.2
)

In [13]:
model = make_model()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True) # Early Stopping to prevent overflow
model.fit(x_train, y_train, epochs=200, callbacks=[early_stopping], validation_data=(x_test, y_test))
model.evaluate(x_test, y_test, verbose=2) 

In [ ]:
model.save("SignLanguage9C991.h5")